In [2]:
!pip install konlpy
!pip install soynlp
!pip install transformers
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-ljoy7kla
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-ljoy7kla
ERROR: You must give at least one requirement to install (see "pip help install")


In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from konlpy.tag import Hannanum
from hanspell import spell_checker
import re
import matplotlib
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import matplotlib.font_manager as fm

## 데이터 불러오기 및 확인

In [4]:
youtube_df = pd.read_csv("preprocessed_emotion_labeled_youtube_0607.csv", lineterminator='\n')
youtube_df[:50]

,car_brand,car_model,comments,likes,related,emotion
0,BMW,1시리즈,"주제 선정 좋아요. 더운 여름 잘 지내세요. 비싼 차, 싼 차, 슈퍼카, 경차 가리...",3,1,0
1,BMW,1시리즈,주차장시승평잘봤네요ㅋ그냥외관이랑실내평만보면편안할듯ㅋㅋ매력적인차네요세컨으로,3,1,0
2,BMW,1시리즈,차 가격도 싸고 디자인도 예쁘고 엄청 좋다,5,1,0
3,BMW,1시리즈,좀끄럽다하셨는데G305시리즈랑어느정도차이가날까요?ㅎㅎ,2,1,0
4,BMW,1시리즈,그나마 전륜으로 바뀌면서 공간이 좀 쓸 만해졌죠. 이 전 세대 1시리즈는 정말......,2,1,1
5,BMW,1시리즈,궁금했던 차량 리뷰해주셔서 감사합니다 작은 차를 좋아해서 그런지 충분히 매력적이네요,4,1,-1
6,BMW,1시리즈,이뿐데출력이아쉽네요.윗급20D에들어가는190마력짜리를디튠한거라면칩튠이나맵핑했을때얼마...,1,1,0
7,BMW,1시리즈,1시리즈도 뭔가 매력적인데ㅜㅜ가격적인 부분도 매력적이고ㅜㅜ!!,5,1,0
8,BMW,1시리즈,와 많이 이뻐졌네요 저도 2016년식 하나 들고 있는데 애착 많이 가는 차에요,3,1,0
9,BMW,1시리즈,1시리즈는 세단은 너무 답답하고 SUV는 너무 클 때 타는 게 맞을 뜻,19,1,1


In [5]:
youtube_df.rename(columns = {'likes\r':'likes'}, inplace = True) # 저는 파일을 읽어오면 likes\r로 columns가 나와서 에러가 나나 싶어소 혹시 바꾸어줬습니다.

In [6]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120488 entries, 0 to 120487
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   car_brand  120488 non-null  object
 1   car_model  120488 non-null  object
 2   comments   120183 non-null  object
 3   likes      120488 non-null  int64 
 4   related    120488 non-null  int64 
 5   emotion    120488 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 5.5+ MB


In [7]:
youtube_df

,car_brand,car_model,comments,likes,related,emotion
0,BMW,1시리즈,"주제 선정 좋아요. 더운 여름 잘 지내세요. 비싼 차, 싼 차, 슈퍼카, 경차 가리...",3,1,0
1,BMW,1시리즈,주차장시승평잘봤네요ㅋ그냥외관이랑실내평만보면편안할듯ㅋㅋ매력적인차네요세컨으로,3,1,0
2,BMW,1시리즈,차 가격도 싸고 디자인도 예쁘고 엄청 좋다,5,1,0
3,BMW,1시리즈,좀끄럽다하셨는데G305시리즈랑어느정도차이가날까요?ㅎㅎ,2,1,0
4,BMW,1시리즈,그나마 전륜으로 바뀌면서 공간이 좀 쓸 만해졌죠. 이 전 세대 1시리즈는 정말......,2,1,1
...,...,...,...,...,...,...
120483,혼다,파일럿,"파일럿 오너입니다. 패밀리카로는 정말 최고입니다. 넓은 공간, 조용하고 묵직한 주행...",3,1,0
120484,혼다,파일럿,제가 아는 현대차 카마스터님과 말투가 너무 비슷하시네요 파일럿 최고죠 할인해줬던 가...,2,1,1
120485,혼다,파일럿,혼다 변속기 버튼은 헷갈리지 않습니다. 직관적입니다. 현대전자식 변속기가 배워야 할...,4,1,0
120486,혼다,파일럿,이전에 F10520D 타 신서 기억합니다. 그 이후에 F10520D를 구매해서 타고...,2,1,0


## 데이터 전처리

In [8]:
# youtube_df['comments'] = youtube_df['comments'].str.replace("[^A-Za-z0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# youtube_df['comments']

In [9]:
# 3036개 - 10분 58초
# 전처리 파일 완성되면 코드 삭제

# import re

# words = youtube_df['comments']

# for i in range(len(words)):
    
#   spelled_sent = spell_checker.check(youtube_df['comments'][i]) # 맞춤법 검사
#   hanspell_sent = spelled_sent.checked # 띄어쓰기 교정
#   result = " ".join(hanspell_sent.split())
#   youtube_df['comments'][i] = result
  
#   print(result)


# youtube_df

In [10]:
# 라벨을 int -> str으로 바꾸기
youtube_df['related'] = youtube_df['related'].astype(str)
youtube_df['emotion'] = youtube_df['emotion'].astype(str)

In [11]:
# 공백으로 된 행 nan 값으로 바꿔주기
youtube_df['comments'] = youtube_df['comments'].replace('', np.nan)

In [12]:
# 공백으로 된 행 갯수
youtube_df['comments'].isnull().sum()

305

In [13]:
# 중립 행 없애기
youtube_df['emotion'] = youtube_df['emotion'].replace('-1', np.nan)

In [14]:
youtube_df['emotion'].isnull().sum()

1943

In [15]:
# nan 값 삭제
youtube_df = youtube_df.dropna(how = 'any')
youtube_df

,car_brand,car_model,comments,likes,related,emotion
0,BMW,1시리즈,"주제 선정 좋아요. 더운 여름 잘 지내세요. 비싼 차, 싼 차, 슈퍼카, 경차 가리...",3,1,0
1,BMW,1시리즈,주차장시승평잘봤네요ㅋ그냥외관이랑실내평만보면편안할듯ㅋㅋ매력적인차네요세컨으로,3,1,0
2,BMW,1시리즈,차 가격도 싸고 디자인도 예쁘고 엄청 좋다,5,1,0
3,BMW,1시리즈,좀끄럽다하셨는데G305시리즈랑어느정도차이가날까요?ㅎㅎ,2,1,0
4,BMW,1시리즈,그나마 전륜으로 바뀌면서 공간이 좀 쓸 만해졌죠. 이 전 세대 1시리즈는 정말......,2,1,1
...,...,...,...,...,...,...
120483,혼다,파일럿,"파일럿 오너입니다. 패밀리카로는 정말 최고입니다. 넓은 공간, 조용하고 묵직한 주행...",3,1,0
120484,혼다,파일럿,제가 아는 현대차 카마스터님과 말투가 너무 비슷하시네요 파일럿 최고죠 할인해줬던 가...,2,1,1
120485,혼다,파일럿,혼다 변속기 버튼은 헷갈리지 않습니다. 직관적입니다. 현대전자식 변속기가 배워야 할...,4,1,0
120486,혼다,파일럿,이전에 F10520D 타 신서 기억합니다. 그 이후에 F10520D를 구매해서 타고...,2,1,0


In [16]:
youtube_df = youtube_df.reset_index()

In [17]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118260 entries, 0 to 118259
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   index      118260 non-null  int64 
 1   car_brand  118260 non-null  object
 2   car_model  118260 non-null  object
 3   comments   118260 non-null  object
 4   likes      118260 non-null  int64 
 5   related    118260 non-null  object
 6   emotion    118260 non-null  object
dtypes: int64(2), object(5)
memory usage: 6.3+ MB


## 브랜드명, 차량 모델명 리스트에 담기

In [18]:
# 브랜드명 리스트에 담기
car_brand_lst = list(youtube_df['car_brand'].value_counts().index)
car_brand_lst

['현대',
 '기아',
 '한국GM',
 '벤츠',
 '르노코리아',
 'BMW',
 '아우디',
 '폭스바겐',
 '쌍용',
 '포드',
 '포르쉐',
 '도요타',
 '볼보',
 '재규어',
 '닛산',
 '혼다',
 '랜드로버',
 '푸조',
 '시트로엥',
 '미쯔비시',
 '롤스로이스',
 '기타',
 '마쯔다',
 '피아트',
 '크라이슬러',
 '사브',
 '다이하쓰']

In [19]:
# 차량 모델 리스트에 담기
car_model_lst = list(youtube_df['car_model'].value_counts().index)
car_model_lst

['기타',
 '트위지',
 '쏘나타',
 'XM3',
 '아반떼',
 '다마스',
 'F150',
 'K5',
 '윈드스타',
 '싼타페',
 '투싼',
 '타이칸',
 '제네시스',
 'S-클래스',
 '보라',
 '캐스퍼',
 '포터',
 'K8',
 '모닝',
 '무쏘',
 'iX',
 '제타',
 'X5',
 '스타렉스',
 'K9',
 '쏘렌토',
 'EV6',
 '아이오닉',
 '레토나',
 '니로',
 '카니발',
 '쉐보레 말리부',
 'SM6',
 '쉐보레 트레일블레이저',
 '코나',
 '아이오닉5',
 '쉐보레 스파크',
 '셀토스',
 '티볼리',
 '스포티지',
 '스팅어',
 '모하비',
 'K7',
 '스토닉',
 '그랜저',
 '체어맨',
 '쉐보레 트래버스',
 '레인지로버 벨라',
 '코란도',
 '오피러스',
 '5시리즈',
 'BB',
 '봉고',
 '4시리즈',
 'G-클래스 (G바겐)',
 '벨로스터',
 '옵티마',
 'Q8',
 '아슬란',
 '투아렉',
 '쉐보레 콜벳',
 'EQA',
 '스타리아',
 '카렌스',
 'A5',
 'i30',
 'A7',
 '클리오',
 '마칸',
 'A6',
 'GLC-클래스',
 'XC90',
 'K3',
 '마치',
 '아테온',
 '어코드',
 'i40',
 'C-클래스',
 'A4',
 '골프',
 '파나메라',
 '캡처',
 'E-Series',
 '토스카',
 '911',
 '티뷰론',
 '알페온',
 '쏘울',
 '206',
 'A-클래스',
 'F-TYPE',
 '파제로',
 '엘란',
 '쉐보레 볼트',
 '수프라',
 'X6',
 'M시리즈',
 'RS Q8',
 '승용차',
 '버스(26인이상)',
 '6시리즈',
 '3시리즈',
 '2008',
 '쉐보레 크루즈',
 'GL-클래스',
 '펠리세이드',
 '머스탱',
 'S7',
 'CLS-클래스',
 'X3',
 'RS7',
 'QM6',
 'S90',
 '쉐보레 콜로라도',
 '제네시스쿠페',
 '쉐보레 

In [20]:
car_lst = list(youtube_df[['car_brand','car_model']].value_counts().index)
car_lst

[('르노코리아', '트위지'),
 ('현대', '기타'),
 ('현대', '쏘나타'),
 ('쌍용', '기타'),
 ('르노코리아', 'XM3'),
 ('현대', '아반떼'),
 ('한국GM', '다마스'),
 ('포드', 'F150'),
 ('기아', 'K5'),
 ('포드', '윈드스타'),
 ('현대', '싼타페'),
 ('포르쉐', '타이칸'),
 ('현대', '투싼'),
 ('현대', '제네시스'),
 ('벤츠', 'S-클래스'),
 ('폭스바겐', '보라'),
 ('현대', '캐스퍼'),
 ('현대', '포터'),
 ('기아', 'K8'),
 ('기아', '모닝'),
 ('쌍용', '무쏘'),
 ('BMW', 'iX'),
 ('폭스바겐', '제타'),
 ('BMW', 'X5'),
 ('현대', '스타렉스'),
 ('기아', 'K9'),
 ('기아', '쏘렌토'),
 ('기아', 'EV6'),
 ('현대', '아이오닉'),
 ('기아', '레토나'),
 ('기아', '니로'),
 ('기아', '카니발'),
 ('한국GM', '쉐보레 말리부'),
 ('르노코리아', 'SM6'),
 ('한국GM', '쉐보레 트레일블레이저'),
 ('현대', '코나'),
 ('현대', '아이오닉5'),
 ('한국GM', '쉐보레 스파크'),
 ('기아', '셀토스'),
 ('쌍용', '티볼리'),
 ('기아', '스포티지'),
 ('기아', '스팅어'),
 ('포드', '기타'),
 ('벤츠', '기타'),
 ('기아', '모하비'),
 ('기아', 'K7'),
 ('기아', '스토닉'),
 ('현대', '그랜저'),
 ('쌍용', '체어맨'),
 ('한국GM', '쉐보레 트래버스'),
 ('랜드로버', '레인지로버 벨라'),
 ('쌍용', '코란도'),
 ('기아', '오피러스'),
 ('BMW', '5시리즈'),
 ('도요타', 'BB'),
 ('기아', '봉고'),
 ('BMW', '4시리즈'),
 ('벤츠', 'G-클래스 (G바겐)'),
 ('현대', '벨로스터'

In [21]:
len(car_lst)

419

In [22]:
youtube_df[(youtube_df['car_brand'].str.contains('벤츠')&youtube_df['car_model'].str.contains('G-클래스 (G바겐)'))]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,index,car_brand,car_model,comments,likes,related,emotion


In [23]:
youtube_df

,index,car_brand,car_model,comments,likes,related,emotion
0,0,BMW,1시리즈,"주제 선정 좋아요. 더운 여름 잘 지내세요. 비싼 차, 싼 차, 슈퍼카, 경차 가리...",3,1,0
1,1,BMW,1시리즈,주차장시승평잘봤네요ㅋ그냥외관이랑실내평만보면편안할듯ㅋㅋ매력적인차네요세컨으로,3,1,0
2,2,BMW,1시리즈,차 가격도 싸고 디자인도 예쁘고 엄청 좋다,5,1,0
3,3,BMW,1시리즈,좀끄럽다하셨는데G305시리즈랑어느정도차이가날까요?ㅎㅎ,2,1,0
4,4,BMW,1시리즈,그나마 전륜으로 바뀌면서 공간이 좀 쓸 만해졌죠. 이 전 세대 1시리즈는 정말......,2,1,1
...,...,...,...,...,...,...,...
118255,120483,혼다,파일럿,"파일럿 오너입니다. 패밀리카로는 정말 최고입니다. 넓은 공간, 조용하고 묵직한 주행...",3,1,0
118256,120484,혼다,파일럿,제가 아는 현대차 카마스터님과 말투가 너무 비슷하시네요 파일럿 최고죠 할인해줬던 가...,2,1,1
118257,120485,혼다,파일럿,혼다 변속기 버튼은 헷갈리지 않습니다. 직관적입니다. 현대전자식 변속기가 배워야 할...,4,1,0
118258,120486,혼다,파일럿,이전에 F10520D 타 신서 기억합니다. 그 이후에 F10520D를 구매해서 타고...,2,1,0


# 브랜드, 차량 모델별로 for문 돌리면서 워드 클라우드 생성


In [24]:
# 폰트 다운
%config InlineBackend.figure_format = 'retina'

!apt -qq -y install fonts-nanum > /dev/null

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

In [25]:
# 자동차 이미지로 출력하기 위헤 흑백이미지를 백터화
from PIL import Image
import numpy as np

img_array = np.array(Image.open('car_image.png').convert('L'))

for i in img_array:
  print(i)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255]
[255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 253 178  77   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0  63 160 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 202  93   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 101 215 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255 255 255 255 255 252 122   9   0   0   0   0   0   0   0   0
   0   0   0   0   0   0  

In [26]:
# 불용어 설정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','듯','것','저','ㅋㅋ','수','ㅎㅎ','ㅋㅋㅋㅋㅋ','ㅋㅋㅋ','ㅋㅋㅋㅋ','ㅋ','나','때','데','지','현대','기아','한국GM','벤츠','르노코리아','BMW','아우디','폭스바겐','쌍용','포드','포르쉐','도요타','볼보','재규어','닛산','혼다','랜드로버',
 '푸조','시트로엥','미쯔비시','롤스로이스','기타','마쯔다','피아트','크라이슬러','사브','다이하쓰','기타','트위지','쏘나타','XM3','아반떼','다마스','F150','K5','윈드스타','싼타페','제네시스','타이칸','투싼','S-클래스','캐스퍼','보라','포터','K8','모닝','제타','iX','X5','무쏘','스타렉스','쏘렌토','EV6','K9','아이오닉','레토나','쉐보레 말리부','카니발',
 '니로','쉐보레 트레일블레이저','SM6','코나','아이오닉5','셀토스','티볼리','쉐보레 스파크','스팅어','스포티지','모하비','K7','그랜저','스토닉','체어맨','쉐보레 트래버스','레인지로버 벨라','코란도','5시리즈','오피러스','BB','벨로스터','봉고','4시리즈','G-클래스 (G바겐)','옵티마','Q8','투아렉','쉐보레 콜벳','EQA','스타리아','아슬란','i30','A5',
 '카렌스','클리오','A7','GLC-클래스','A6','마칸','XC90','K3','아테온','어코드','마치','i40','C-클래스','A4','골프','파나메라','캡처','E-Series','토스카','911','티뷰론','쏘울','A-클래스','알페온','파제로','206','F-TYPE','수프라','엘란','쉐보레 볼트','X6','3시리즈','버스(26인이상)','M시리즈','2008','RS Q8','GL-클래스','승용차',
 '머스탱','펠리세이드','6시리즈','쉐보레 크루즈','X3','CLS-클래스','S7','RS7','QM6','S90','레인지로버 이보크','제네시스쿠페','쉐보레 트랙스','쉐보레 콜로라도','쉐보레 카마로','아카디아','트럭(4.5톤이상)','엔터프라이즈','DS7','Z4','T-Roc','포니','1시리즈','SLC-클래스','e-tron','XC60','GLS-클래스','오딧세이','C30','고스트','엘란트라','CR-V',
 '프리우스','엑스트렉','트럭','티구안','XF','로체','랜서 에볼루션','티코','A8','SL-클래스','쉐보레 이쿼녹스','C3','XC40','디스커버리 스포츠','CLA-클래스','마크X','Q3','세레스','XJ-6','I-PACE','Q2','AMG GT','R8','엑셀','다이너스티','2시리즈','갤로퍼','디펜더','넥쏘','X7','카이엔','조에','SM7','Q5','S70','파사트','포르테','맥스크루즈',
 '박스터','CL-클래스','레이','CLK-클래스','크레도스','8시리즈','시로코','208','스카이라인','RAV 4','S6','X4','디스커버리','GT-R','86','508','CX-5','MX-5 MIATA','GLE-클래스','트럭(2.5~4.5톤)','RS5','리오','에쿠스','쉐보레 올란도','레이스','3008','던','A3','EOS','XJ-8','406','렉스턴','리지라인','알티마','파일럿','쉐보레 캡티바',
 '버스(16~36인)','GLB-클래스','마이티','V50','DS4','시에나','S60','i8','쏠라티','9-3','쉐보레 임팔라','액티언''트레일러','S-TYPE','SM5','C5','QM3','델솔','마테리아','SLK-클래스','라보','S8','매그너스','아발론','이스타나','124','투스카니','브로엄','리베로','엑센트','프론티어','레인지로버','Q7','카이맨','리프','DS3','X1','몬데오','비스토',
 'A1','500','시빅','S3','V90','스프린터','XJR','캐러밴','마스터','쉐보레 아베오','5008','SM3','레인지로버 스포츠','X2','i3','XE','SQ5','C4','레인저','V60','E-PACE','패스파인더','SQ8','EQC','카이런','타이탄','WiLL','코펜','X-Trail','S660','FJ 크루져','그란투리스모 (GT)','N Box','XKR''캠리','마르샤','QM5','캐피탈','로디우스','테라칸',
 '포텐샤','이클립스','200','308','휘가로','베리타스','F-PACE','500X ','스텔라','라세티','그레이스','폴로','쿠가','HR-V','TT','NV','익스플로러','비틀','RVR','싼타모','세피아','마티즈','V40','GLA-클래스','툰드라','파쏘','익스피디션','맥시마','대우버스','370Z','9-5','페이톤','V-클래스','B-클래스 (MY B)','트라제XG','레전드','클릭','엘그란드',
 '7시리즈','스펙트라','익스플로러 스포츠트랙','X-TYPE','무라노','CC','엑시언트','하이랜더','207','S4','G2X','트라고','C70','트랜짓','타코마','베르나','솔라라','프라이드','쎄라토','엘리멘트','9000','XC70','900','300C','PT 크루저','S-MAX','베라크루즈','르망','라비타','윈스톰','레조','라노스','프레스토','티아나','센트라','607','타우너',
 'TTS','307','큐브','프런티어','M-클래스','벤자','소버린','미라코코아','아웃랜더','쥬크','프린스','407','GLK-클래스','젠트라','굴삭기','크로스파이어','RCZ','S5','XJ','파이브 헌드레드','그랜드 보이저','프리랜더','이스케이프','퓨전','미라지노','S40','프레지오','카스타','모코','타운 앤 컨트리','랜서','S80','R-클래스','올로드 콰트로','스테이츠맨','크로스투어',
 '코롤라','퍼시피카','토러스','세브링','프리몬트','DS5','인사이트','포커스','로그','차','단점','장점','근데','이거','분','비','저거','정도','나','말','ㅎ','되','때문','내','ㅎㅎㅎㅎㅎ','지금','개','점','타고','편','저희','뭐','저차','그','우','바','만','라','님','으','ㄷㄷㄷ','ㅠ','ㅜ','등','얘','보','그거','ㅎㅎㅎㅎ','ㅎㅎ저','기','ㅠㅠㅠ','타','후',
 '시발','쌍','차','것','수','진짜','코','리','때','적','ㅎㅎ','지','정도','작','라','오','이젠','보','바','만','쪽','나름','안되서','내','안','이거','대','뿐','ㅠ','두','되길','뭔가','그걸','드','왜이렇게','분','ㄷㄷ','조금더','평소','ㄷ','행','되겼음','어찌됫건','유','하지만지금까지','님','전','레','G80','ㅎㅎㅎ','쉐보레','ㅠㅠ','식','G90','G70','전',
 'GV80','이번','소나타','GV70','거','마스','위','형','중','엠','우리','시','2','3','세','와우','^ㅎ^ㅎ','변','터','솔','추','모','사실','아우','노','번','무','4.','1.','년','김한용','3.','십','^ㅋ','^ㅎ','^','5','2.','면','1.','^ㅎ^ㅎ^ㅎ','등등','용','제','^ㅋ^ㅋ^ㅋ','^ㅋ^ㅋ^ㅋ^ㅋ^ㅋ','어디','조','ㅂ','닥','징','란','거에요','그다음','이하',
 '음']

# 고려할 단어
# '리뷰', '기자님','영상'

han = Hannanum()
tokenizer = Tokenizer()

In [27]:
# 긍정

positive_han_nouns = []


for i in range(len(car_lst)):
  car_youbute_df = youtube_df[(youtube_df['car_brand'].str.contains(car_lst[i][0])&youtube_df['car_model'].str.contains(car_lst[i][1]))]

  positive_han_nouns = []

  # 감성 분류 긍정(0)이면 -> 워드 클라우드 생성
  positive_df = car_youbute_df[(car_youbute_df['emotion'].str.contains('0'))]
  print(car_lst[i][0], car_lst[i][1])
  print(len(positive_df))
  try:
    for p_comment in positive_df['comments']:
      tokenized_sentence_p = han.nouns(p_comment)
      stopwords_removed_sentence_p = [word for word in tokenized_sentence_p if not word in stopwords] # 불용어 제거
      positive_han_nouns.append(stopwords_removed_sentence_p)

      # 글자수 한자리 단어, 'ㅋ'이 포함된 단어은 불용어 사전에 넣기
    #   for j in stopwords_removed_sentence_p:
    #     if len(j) <= 1 or 'ㅋ' or 'ㅠ' or 'ㅡ' or 'ㅎ' in j:
    #       stopwords.append(j)
    # print(stopwords)

    word_list_p = sum(positive_han_nouns, [])
    count_p = Counter(word_list_p)
    word_count_p = dict(count_p.most_common())
    print(word_count_p)

    # 로컬에 있는 폰트를 사용할 경우, 폰트의 경로를 font_path에 추가 해주시면 됩니다.
    wc = WordCloud(font_path=fontpath, background_color = 'white',colormap=matplotlib.cm.ocean, width=800, height=800, min_font_size = 8, max_font_size=120, prefer_horizontal = True, mask=img_array)
    cloud_p = wc.fit_words(word_count_p)
    cloud_p.to_image()
    cloud_p.to_file(f'{car_lst[i][0]}+{car_lst[i][1]}+긍정.jpg')
  except:
    print(car_lst[i][0], car_lst[i][1],'/n')
    pass

Output hidden; open in https://colab.research.google.com to view.

In [196]:
# 부정

nagative_han_nouns = []
# stopwords_removed_sentence_n = []

for i in range(len(car_lst)):
  car_youbute_df = youtube_df[(youtube_df['car_brand'].str.contains(car_lst[i][0])&youtube_df['car_model'].str.contains(car_lst[i][1]))]

  nagative_df = car_youbute_df[(car_youbute_df['emotion'].str.contains('1'))]
  print(car_lst[i][0], car_lst[i][1])
  print(len(nagative_df))

  try:
    for n_comment in nagative_df['comments']:
      tokenized_sentence_n = han.nouns(n_comment)
      # print(tokenized_sentence_n, n_comment)
      # print(tokenized_sentence_n, n_comment)

      # for word in tokenized_sentence_n: 
      #   if len(word) <= 1:
      #     stopwords.append(word)
      #   else:
      #     if word not in stopwords:
      #       stopwords_removed_sentence_n.append(word) 
      stopwords_removed_sentence_n = [word for word in tokenized_sentence_n if not word in stopwords] # 불용어 제거
      nagative_han_nouns.append(stopwords_removed_sentence_n)


      # print(nagative_han_nouns)

      # 글자수 한자리 단어, 'ㅋ'이 포함된 단어은 불용어 사전에 넣기
      # for z in stopwords_removed_sentence_n:
      #   if len(z) <= 1 or 'ㅋ' or 'ㅠ' or 'ㅡ' or 'ㅎ' in j:
      #     stopwords.append(z)

    word_list_n = sum(nagative_han_nouns, [])
    count_n = Counter(word_list_n)
    word_count_n = dict(count_n.most_common())
    print(word_count_n)

    # 로컬에 있는 폰트를 사용할 경우, 폰트의 경로를 font_path에 추가 해주시면 됩니다.
    wc = WordCloud(font_path=fontpath, background_color = 'white',colormap=matplotlib.cm.gist_heat, min_font_size = 8, max_font_size=120, width=800, height=800, prefer_horizontal = True, mask=img_array)
    cloud_n = wc.fit_words(word_count_n)
    cloud_n.to_image()
    cloud_n.to_file(f'{car_lst[i][0]}+{car_lst[i][1]}+부정.jpg')
  except:
    print(car_lst[i][0], car_lst[i][1],'/n')
    pass

       Unnamed: 0  index car_brand car_model  \
39878       39878  70175     르노코리아       트위지   
39881       39881  70178     르노코리아       트위지   
39883       39883  70180     르노코리아       트위지   
39885       39885  70182     르노코리아       트위지   
39891       39891  70193     르노코리아       트위지   
...           ...    ...       ...       ...   
42598       42598  75881     르노코리아       트위지   
42599       42599  75882     르노코리아       트위지   
42600       42600  75884     르노코리아       트위지   
42601       42601  75887     르노코리아       트위지   
42602       42602  75888     르노코리아       트위지   

                                                comments  likes related  \
39878  이미 셱셱버거를 비롯해서 몇몇 군데에서는 배달용으로 사용하고 있는 모양이던데 아무래...     91       1   
39881       이 차가 바로 초보운전 대신 질문금지 스티커를 붙이고 다닌다는 그 차군요 ㅎㅎㅎ    223       1   
39883  4륜차 변형이라고 보면 불편한 점이 너무 많다고 봅니다 물론 가격을 보면 무리한 요...      2       1   
39885  비 오면 대박겨울에도 대박매연도 대박후진 주차도 보안장치가 제로창문좀 달아만 줘도 ...      1       1   
39891                얼마전에 트위지 실제로 봤는데매우 작고 매우 눈에

KeyboardInterrupt: ignored

In [24]:
car_youbute_df = youtube_df[(youtube_df['car_brand'].str.contains(car_lst[0][0])&youtube_df['car_model'].str.contains(car_lst[0][1]))]
nagative_df = car_youbute_df[(car_youbute_df['emotion'].str.contains('1'))]

for i in nagative_df['comments']:
    tokenized_sentence_n = han.nouns(i)
    print(tokenized_sentence_n)



['셱셱버거', '비롯', '몇몇', '군데', '배달용', '사용', '모양', '오토바이', '배달원분', '생각']
['차', '초보운전', '대', '질문금지', '스티커', '차', 'ㅎㅎㅎ']
['트위지', '비교', '영상', '창문', '에어컨', '히터', '사양', '것', '데', '실제', '운용면', '어떨지', '궁금']
['4륜차', '변형', '불편한', '점', '가격', '무리한', '요구', '생각되지만2륜차', '변형', '우천시', '수화물', '무게', '경우', '경우', '이륜차', '대용', '이용', '수']
['비', '대박겨울', '대박매연', '대박후진', '주차', '보안장치', '제로창문좀', '연비', '나']
['얼마전', '트위', '봤는데매우', '눈']
['마트', '판매', 'D2', '전기차', '궁금']
['외국', '경우', '자녀', '통학', '때문', '차', '두대', '운용', '2인승', '사용', '용도']
['겨울', '절대못타', '차네욬ㅋㅋ']
['저', '유명한', '김숙', '차', '에어컨', '안나오', '히터', '창문', '전기충전', '충전소', '두명이서', '개', '빡세다는', '유명한', '차', 'ㅋㅋㅋㅋㅋ', 'ㅋㅋㅋㅋㅋ', 'ㅋㅋㅋㅋㅋ']
['독', '청소용차네요최고', '단점', '창문이', '것', '에어컨', '없다는점', '창문', '슝슝', '들어온다는점']
['디자인', '강조', '차', '쩌']
['영상', '전기차', '단점', '생각해봤습니다1황사미세먼지', '된다2비오', '에어컨', '때문', '검색', '유리', '열', '김', '어느정도', '수', '있다곤', '하나', '배터리소모', '것', '예상됨3어차피', '220V', '충전', '수', '주차장', '전기', '마음', '수', '전기차', '이동식충전기', '220V', '충전', '수', '남의전기', '본인', '명의', '요금', '시스템', '트위지

In [23]:
r = '시골 동네 회관에 어르신들 타고 다니기 좋은 것 같은데그마저도 안되는게시티백  오토바이나경차 마티즈 정도급에서 해결됨마티즈 정도급에서 해결되는데 저차의 문제는 승하차 마져 불편해보임특히 뒷자석걍 마티즈 조수석이 백번 편함'
han.nouns(r)

['시골',
 '동네',
 '회관',
 '것',
 '같은데그마저',
 '안되는게시티백',
 '오토바이나경차',
 '마티즈',
 '정도급',
 '해결됨마티즈',
 '정도급',
 '해결',
 '저차',
 '문제',
 '승하차',
 '마져',
 '불편해보임특히',
 '뒷자석걍',
 '마티즈',
 '조수석',
 '백번']

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

han = Hannanum()
tokenizer = Tokenizer()

positive_han_nouns = []
nagative_han_nouns = []

car_youbute_df = labeling_df[(labeling_df['new_label'].str.contains('0'))]

# 감성 분류 긍정(0)이면 -> 워드 클라우드 생성
positive_df = car_youbute_df[(car_youbute_df['label'].str.contains('0'))]
for p_comment in positive_df['comments']:
    tokenized_sentence_p = han.nouns(p_comment)
    stopwords_removed_sentence_p = [word for word in tokenized_sentence_p if not word in stopwords] # 불용어 제거
    positive_han_nouns.append(stopwords_removed_sentence_p)
    # print(positive_han_nouns)

    # 글자수 한자리 단어, 'ㅋ'이 포함된 단어은 불용어 사전에 넣기
    for j in stopwords_removed_sentence_p:
      if len(j) <= 1 or 'ㅋ' or 'ㅠ' or 'ㅡ' or 'ㅎ' in j:
        stopwords.append(j)
    # print(stopwords)

    # tokenizer.fit_on_texts(positive_han_nouns)
    # positive_dic = tokenizer.word_index

word_list_p = sum(positive_han_nouns, [])
count_p = Counter(word_list_p)
word_count_p = dict(count_p.most_common())
print(word_count_p)

# 로컬에 있는 폰트를 사용할 경우, 폰트의 경로를 font_path에 추가 해주시면 됩니다.
wc = WordCloud(font_path=fontpath, background_color = 'white',colormap=matplotlib.cm.Reds,  max_words=100, width=800, height=800, prefer_horizontal = True, mask=img_array)
cloud_p = wc.fit_words(word_count_p)
cloud_p.to_image()
cloud_p.to_file('유튜브 댓글 워드 클라우드 긍정.jpg')


nagative_df = car_youbute_df[(car_youbute_df['label'].str.contains('1'))]
# print(nagative_df,len(nagative_df))
for n_comment in nagative_df['comments']:
    tokenized_sentence_n = han.nouns(n_comment)
    # print(tokenized_sentence_n, n_comment)
    stopwords_removed_sentence_n = [word for word in tokenized_sentence_n if not word in stopwords] # 불용어 제거
    nagative_han_nouns.append(stopwords_removed_sentence_n)
    # print(nagative_han_nouns)

    # 글자수 한자리 단어, 'ㅋ'이 포함된 단어은 불용어 사전에 넣기
    for z in stopwords_removed_sentence_n:
      if len(z) <= 1 or 'ㅋ' or 'ㅠ' or 'ㅡ' or 'ㅎ' in j:
        stopwords.append(z)
    # tokenizer.fit_on_texts(nagative_han_nouns)
    # nagative_dic = tokenizer.word_index

word_list_n = sum(nagative_han_nouns, [])
count_n = Counter(word_list_n)
word_count_n = dict(count_n.most_common())
print(word_count_n)

# 로컬에 있는 폰트를 사용할 경우, 폰트의 경로를 font_path에 추가 해주시면 됩니다.
wc = WordCloud(font_path=fontpath, background_color = 'white',colormap=matplotlib.cm.Blues,  max_words=100, width=800, height=800, prefer_horizontal = True)
cloud_n = wc.fit_words(word_count_n)
cloud_n.to_image()
cloud_n.to_file('유튜브 댓글 워드 클라우드 부정.jpg')



{'아쉬움': 7, '원터치': 4, '사용': 4, '창문': 3, '하체': 3, '운전석': 3, '택시': 2, '그랜저': 2, 'ㅎㅎ': 2, '전면': 2, '리뷰': 2, '전동트렁크': 2, '확실': 2, '필요': 2, '녹화': 2, '확인': 2, '캠리': 2, '원가절감': 2, '접근': 2, '손': 2, '엔진': 2, '애매': 2, '모습': 2, '분위기': 2, '최악': 2, '본인': 2, '보수적': 2, '최소': 2, '20대': 2, '이상': 2, '찻값': 2, '허리': 2, '충격': 2, '라이트': 2, '페이스리프트': 2, '1': 2, '시': 2, '3': 2, '4': 2, '마감': 2, '부재': 2, '5': 2, '처음': 2, '요소': 2, '삼각': 2, '자동': 2, '전진': 2, '후진': 2, '리모트': 2, '파킹': 2, '주차장': 2, '아방': 2, '좌회전': 2, '브레이크': 2, '보류': 2, '가속성능': 2, '이슈': 2, '1순위': 2, '2순위': 2, '3순위': 2, '일단': 2, '레일': 2, '우드': 2, '무난': 2, '대형': 2, '스포츠': 2, '안정감': 2, '레버': 2, '단종하길': 1, '진짜': 1, '소나타': 1, '이름': 1, '차': 1, '내부': 1, '고급': 1, '지': 1, '신형': 1, '어색': 1, '만큼': 1, '디자인': 1, '앞': 1, '출시': 1, '정도': 1, '품질': 1, '생산': 1, '것': 1, '감성': 1, '쏘렌토': 1, '체': 1, '대형급': 1, '사이즈': 1, '더라고요': 1, 'ㄷ': 1, '그때': 1, '칼': 1, '시승': 1, '현대자동차': 1, '발전': 1, '거': 1, '뭔가': 1, '2프': 1, '부족한': 1, '때문일까ㅜㅜ': 1, '기술': 1, '집합소이네욛': 1, '뉴': 1, '라이즈': 1, 

# 파일 저장 연습하기

In [ ]:
labeling_df = pd.read_csv("labeling.csv", lineterminator='\n')
labeling_df[:50]

,Unnamed: 0,comments,label,likes,new_label,check
0,0,상위모델로 자연스럽게 유도하는 것 같습니다. 그랜저 ig부터는 둥글둥글하고 중후한 ...,0,1,1,X
1,1,확실히 깔끔하고 직관적인 리뷰,1,3,1,O
2,2,"현기가 클래스별 특징들이 점차 뚜렷해지는 모습입니다. 디자인도 좋아지고, 성능이 강...",0,1,1,X
3,3,루프부터 트렁크 끝단까지 라인 미쳤다 ㄷㄷ,0,2,1,X
4,4,옥스포드 블루칼라가 확실히 좋아 보입니다. 그레이 계열은 소나타의 젊음을 상쇄시키고...,0,2,1,X
5,5,기존 뉴라이즈가 뒷좌석 겁나 넓은거였지만nd8에서 줄은건 차 성격이 바뀐것같네요. ...,0,1,1,X
6,6,택시 단종하길 진짜 잘했네요 솔직히 저 '소나타'라는 이름없이 그냥 차로만 봐서는 ...,0,2,0,O
7,7,덕분에 시승 너무 잘 보고 있습니다. 감사합니다.,1,1,0,X
8,8,젊은 감성이네요 ㅎㅎ 쏘렌토 풀체인지도 기대 됩니다 거의 대형급 사이즈 더라구요 ㄷ...,0,1,0,O
9,9,지금은 20대중반이고 크루즈끌지만 30대되면 저것보다 좋은게 나올꺼고 그걸 반드시 ...,0,4,1,X


In [ ]:
labeling_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3037 entries, 0 to 3036
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3037 non-null   int64 
 1   comments    3037 non-null   object
 2   label       3037 non-null   object
 3   likes       3037 non-null   int64 
 4   new_label   3037 non-null   object
 5   check       3037 non-null   object
dtypes: int64(2), object(4)
memory usage: 142.5+ KB


In [ ]:
labeling_df['comments'] = labeling_df['comments'].str.replace("[^A-Za-z0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
labeling_df['comments']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


0       상위모델로 자연스럽게 유도하는 것 같습니다 그랜저 ig부터는 둥글둥글하고 중후한 느...
1                                        확실히 깔끔하고 직관적인 리뷰
2       현기가 클래스별 특징들이 점차 뚜렷해지는 모습입니다 디자인도 좋아지고 성능이 강조된...
3                                 루프부터 트렁크 끝단까지 라인 미쳤다 ㄷㄷ
4       옥스포드 블루칼라가 확실히 좋아 보입니다 그레이 계열은 소나타의 젊음을 상쇄시키고 ...
                              ...                        
3032    언제적 6단 이냐 요새 보통 7단 8단 거의 기본 적용인데6단이라니ZF미션좀 사다 ...
3033                                     잘나봣자ㅡ소나타여그랜저 밑이지
3034                               네비나  저기 각도가 저기 뭐니 균형좀 
3035    코너돌때 일부러 핸들을 흔드세요 코너돌릴때 핸들질이 좀 우습네요가만히 꺾은채 잡고있...
3036                                   이제 개는 안 타고 소나 타는군요
Name: comments, Length: 3037, dtype: object

In [ ]:
for i in labeling_df['comments']:
  print(i)

상위 모델로 자연스럽게 유도하는 것 같습니다 그랜저 ig부터는 둥글둥글하고 중후한 느낌이라면 신형 쏘나타 이하는 젊은 느낌이면서 호불호가 갈리는 느낌입니다
확실히 깔끔하고 직관적인 리뷰
현기가 클래스별 특징들이 점차 뚜렷해지는 모습입니다 디자인도 좋아지고 성능이 강조된 모델은 성능이 강화되고 있고 전반적으로 발전하는 모습이 좋네요 잘 보고 갑니다
루프부터 트렁크 끝단까지 라인 미쳤다 ㄷㄷ
옥스퍼드 블루칼라가 확실히 좋아 보입니다 그레이 계열은 소나타의 젊음을 상쇄시키고 검정과 흰색은 차를 그저 무난하게 만들고 노란색과 빨간색은 좀 튀지만 이번 소나타의 디자인에 잘 어울리네요
기존 뉴 라이즈가 뒷좌석 겁나 넓은 거였지만 nd8에서 줄은 건 차 성격이 바뀐 것 같네요 나중에 25터보 나오면 가격이나 FF 특상상 별로 안 팔릴 것 같긴 한데 ㅋㅋ
택시 단종하길 진짜 잘했네요 솔직히 저 소나타라는 이름 없이 그냥 차로만 봐서는 정말 잘빠지고 내부도 고급 지고 크기도 그렇고 신형 그랜저라고 해도 어색하지 않을 만큼 각 잡고 디자인 잘했네요 앞으로 택시 출시 없이 저 정도 품질로 계속 생산한다면 그랜저만큼 잘 팔릴 것 같습니다
덕분에 시승 너무 잘 보고 있습니다 감사합니다
젊은 감성이네요 ㅎㅎ 쏘렌토 풀체인지도 기대됩니다 거의 대형급 사이즈 더라고요 ㄷ 그때도 칼같이 시승 부탁드립니다 ㅎㅎ
지금은 20대 중반이고 크루즈 끌지만 30대 되면 저것보다 좋은 게 나올 거고 그걸 반드시 사야지 하 진짜 너무 이쁘다 차
현대자동차 정말 많이 발전하고 갈수록 멋지고 좋아지는 건 알겠는데 항상 뭔가 2프로 부족한 느낌은 도대체 왜 때문일까ㅜㅜ
캬이건 기술 집합소이네요ㄷ
뉴 라이즈 보고 디자인 산으로 가나 했는데 디자인 잘 빠졌네요 ㅋㅋㅋ 제대로 된 선택과 집중을 한거 같아요
문제는 저 정도 옵션 누리려면 3500 넘어간다는 사실 ㄷㄷ
와 디자인이랑 실내 너무 고급스럽네요
전면은 옛날 투스카니가 떠오르고 측면은 괜찮고 후면은 삼가게 떼에 어울리는 디자인 차가 작아 보인다 그렌저도 전면

In [ ]:
car_youbute_df = labeling_df[(labeling_df['new_label'].str.contains('0'))]

nagative_df = car_youbute_df[(car_youbute_df['label'].str.contains('1'))]
print(nagative_df,len(nagative_df))
for n_comment in nagative_df['comments']:
  print(n_comment)

      Unnamed: 0                                           comments label  \
7              7                          덕분에 시승 너무 잘 보고 있습니다 감사합니다     1   
16            16             드론이 추가되니 영상이 더 다양해져서 좋네요ㅎㅎ 좋은 영상 감사합니다     1   
28            28  기다렸습니다 다른 시승기가 몇 개 먼저 뜨긴 했는데 역시 한기자님 영상을 봐야 시승...     1   
29            29        다른 미디어는 공동 시승행사였는데 한상기 채널에서는 단독 시승기네요 섭외력 굿     1   
32            32                            수고하셨습니다 재밌게 시승 보고 가겠습니다     1   
...          ...                                                ...   ...   
3026        3026                       저거 과태료 부과해야 하는 거 아닌가요 190이라니     1   
3027        3027                              아직 신차라 그럼 몇만 타면 딸딸이 됨     1   
3028        3028  스마트 스트림 엔진이라고 해도 결국엔 예전 gdi 직분사 엔진이죠 신형 케 3가 m...     1   
3029        3029  신봉자들 많아서 한마디 쓴다 기자가 차를 알면 뭘 알겠냐 운전직도 아니고 차는 내가...     1   
3036        3036                                 이제 개는 안 타고 소나 타는군요     1   

      likes new_label check  
7         1         0     X  
16        1    

In [ ]:
from PIL import Image

img = Image.open('noun-car-1019443.png')
img_array = np.array(img)

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

han = Hannanum()

youtube_review_han_nouns = []

if labeling_df['label'] == '0':

  for i in hanspell_sent_lst:
      tokenized_sentence = han.nouns(i)
      stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
      youtube_review_han_nouns.append(stopwords_removed_sentence)

      # 글자수 한자리 단어, 'ㅋ'이 포함된 단어은 불용어 사전에 넣기
      for j in stopwords_removed_sentence:
        if len(j) <= 1 or 'ㅋ' or 'ㅠ' or 'ㅡ' or 'ㅎ' in j:
          stopwords.append(j)

  print(stopwords)

  print(youtube_review_han_nouns)

  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(youtube_review_han_nouns)
  count_dic = tokenizer.word_index

  print(count_dic)


ValueError: ignored

In [ ]:
|

In [ ]:
# 불용어 설정

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

han = Hannanum()

youtube_review_han_nouns = []

for i in hanspell_sent_lst:
    tokenized_sentence = han.nouns(i)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    youtube_review_han_nouns.append(stopwords_removed_sentence)

    # 글자수 한자리 단어, 'ㅋ'이 포함된 단어은 불용어 사전에 넣기
    for j in stopwords_removed_sentence:
      if len(j) <= 1 or 'ㅋ' or 'ㅠ' or 'ㅡ' or 'ㅎ' in j:
        stopwords.append(j)

print(stopwords)

print(youtube_review_han_nouns)

['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '상위', '모델', '자연', '유도', '것', '그랜저', '둥글둥글하', '중후', '느낌', '신형', '이하', '느낌', '호불호', '리', '느낌', '깔끔', '직관적', '리뷰', '현기', '클래스별', '특징들', '뚜렷', '모습', '디자인', '성능', '강조', '성능', '강화', '전반적', '발전', '모습', '루프', '트렁크', '끝단', '라', 'ㄷㄷ', '옥스퍼드', '블루칼라', '그레이', '계열', '소나타', '젊음', '상쇄', '검정', '흰색', '차', '무난', '노란색', '빨간색', '이번', '소나타', '기존', '뉴', '라이즈', '뒷좌석', '겁', '거', 'nd8', '줄', '거', '성격', '나중', '25터보', '가격', '특상상', '같긴', 'ㅋㅋ', '택시', '단종하길', '진짜', '이름', '내부', '고급', '지', '어색', '만큼', '앞', '택시', '출시', '정도', '품질', '생산', '덕분', '시승', '감사', '감성', 'ㅎㅎ', '쏘렌토', '체', '대형급', '사이즈', '더라고요', 'ㄷ', '그때', '칼', 'ㅎㅎ', '지금', '20대', '중반', '크루즈', '30대', '저것', '그걸', '현대자동차', '뭔가', '2프', '부족한', '때문일까ㅜㅜ', '기술', '집합소이네욛', '산', 'ㅋㅋㅋ', '선택', '집중', '한거', '문제', '옵션', '누리', '3500', '사실', '실내', '전면', '옛날', '투스카니', '측면', '후', '떼', '그렌저', '전면', '형태', '데', '그', '기대', '드론', '추가', '영상', '다양', '좋네욯ㅎ', '영상', '개인적', '다음', '8단', '미션', '텐데', '옆',

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(youtube_review_han_nouns)
count_dic = tokenizer.word_index

print(count_dic)

{'느낌': 1, '목소리': 2, '모습': 3, '성능': 4, '소나타': 5, '거': 6, '택시': 7, 'ㅎㅎ': 8, '전면': 9, '영상': 10, '점': 11, '시승기': 12, 'g70': 13, '구독자': 14, '전동트렁크': 15, '99프': 16, '아반떼': 17, '적응': 18, '차주들': 19, '제네시스': 20, '신차': 21, '상위': 22, '모델': 23, '자연': 24, '유도': 25, '것': 26, '그랜저': 27, '둥글둥글하': 28, '중후': 29, '신형': 30, '이하': 31, '호불호': 32, '리': 33, '깔끔': 34, '직관적': 35, '리뷰': 36, '현기': 37, '클래스별': 38, '특징들': 39, '뚜렷': 40, '디자인': 41, '강조': 42, '강화': 43, '전반적': 44, '발전': 45, '루프': 46, '트렁크': 47, '끝단': 48, '라': 49, 'ㄷㄷ': 50, '옥스퍼드': 51, '블루칼라': 52, '그레이': 53, '계열': 54, '젊음': 55, '상쇄': 56, '검정': 57, '흰색': 58, '차': 59, '무난': 60, '노란색': 61, '빨간색': 62, '이번': 63, '기존': 64, '뉴': 65, '라이즈': 66, '뒷좌석': 67, '겁': 68, 'nd8': 69, '줄': 70, '성격': 71, '나중': 72, '25터보': 73, '가격': 74, '특상상': 75, '같긴': 76, 'ㅋㅋ': 77, '단종하길': 78, '진짜': 79, '이름': 80, '내부': 81, '고급': 82, '지': 83, '어색': 84, '만큼': 85, '앞': 86, '출시': 87, '정도': 88, '품질': 89, '생산': 90, '덕분': 91, '시승': 92, '감사': 93, '감성': 94, '쏘렌토': 95, '체': 96, '대형급': 97, '사이즈': 9